In [2]:
import psycopg2
import requests
import json
import os
from psycopg2.extras import Json

#--------------------Review

url = "https://api.nytimes.com/svc/mostpopular/v2/viewed/30.json?api-key=eX4qBYG8AUGOanGSvqUGJFCi5hUoxSSB"

payload={}
headers = {
  'Accept': 'application/json'
}

response_viewed = requests.request("GET", url, headers=headers, data=payload)

#---------------------emailed
url = "https://api.nytimes.com/svc/mostpopular/v2/emailed/30.json?api-key=eX4qBYG8AUGOanGSvqUGJFCi5hUoxSSB"

payload={}
headers = {
  'Accept': 'application/json'
}

response_email = requests.request("GET", url, headers=headers, data=payload)

url = "https://api.nytimes.com/svc/movies/v2/reviews/all.json?offset=0&order=by-publication-date&api-key=eX4qBYG8AUGOanGSvqUGJFCi5hUoxSSB"

payload={}
headers = {
    'Accept': 'application/json'
}

response_movie = requests.request("GET", url, headers=headers, data=payload)

print("API done")

API done


In [5]:
#-----Password hidden
f=open("PW.txt", "r") 
userPw=[f.readline()]
response_movie

<Response [503]>

In [4]:
data_viewed = response_viewed.json()
data_email = response_email.json()
data_movies = response_movie.json()
#data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data_viewed_res = data_viewed['results']

data_email_res = data_email['results']

data_movies_res = data_movies['results']

In [ ]:
#------------------------SQL Connection

conn = psycopg2.connect(host="mds-dsi-db.postgres.database.azure.com",
                        port="5432",
                        database="nyt_import",
                        user="ds22m017",
                        password=str(userPw[0]),
                        connect_timeout=3)
cur = conn.cursor()

print("con done")

In [13]:
#-----------------------CREATE TABLES


tables = ['ds22m017_emailed','ds22m017_viewed','ds22m017_movies']
for table in tables:
    cur.execute(f"DROP TABLE IF EXISTS {table} CASCADE;")
    cur.execute(f"CREATE TABLE IF NOT EXISTS {table} (id serial, body JSONB);")
conn.commit()


print("create done")

create done


In [14]:

for res in data_email_res:
    val=Json(res)
    cur.execute(f"INSERT INTO ds22m017_emailed (body) VALUES (%s)", [val])
conn.commit() 
    

for res in data_viewed_res:
    val=Json(res)
    cur.execute(f"INSERT INTO ds22m017_viewed (body) VALUES (%s)", [val])
conn.commit()

for res in data_movies_res:
    val=Json(res)
    cur.execute(f"INSERT INTO ds22m017_movies (body) VALUES (%s)", [val])
conn.commit()
    